<a href="https://colab.research.google.com/github/monya-9/deep-learning-practice/blob/main/01_mlp_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch로 MLP 구현 (MNIST 분류기)




In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
# 1. 디바이스 설정 (GPU 있으면 GPU 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# 2. MNIST 데이터셋 로딩
transform = transforms.ToTensor()

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)





1.   MNIST 데이터를 torchvision에서 받아와서 텐서로 변환
2.   학습용(train), 테스트용(test) 데이터셋 분리
3.   DataLoader는 미니배치 단위로 데이터를 묶어줘서 학습 편리하게 함





In [ ]:
# 3. MLP 모델 정의
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )
    def forward(self, x):
        return self.layers(x)

model = MLP().to(device)



1.   28x28 크기 이미지를 1차원 벡터로 펼침
2.   두 개의 은닉층(128, 64)과 ReLU 활성화 함수 사용
3.   마지막 출력층은 10개 클래스(0~9 숫자)
4.   모델을 device에 올려서 GPU/CPU에서 연산 가능하도록 함




In [ ]:
# 4. 손실 함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)



1.   CrossEntropyLoss: 분류 문제에서 가장 많이 쓰는 손실 함수
2.   SGD 옵티마이저: 확률적 경사 하강법, 학습률 0.01 지정



In [ ]:

# 5. 학습 함수
def train(model, loader):
    model.train()
    total_loss = 0
    correct = 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = output.argmax(dim=1)
        correct += (preds == y).sum().item()

    avg_loss = total_loss / len(loader)
    accuracy = correct / len(loader.dataset)
    print(f"Train Loss: {avg_loss:.4f} | Train Accuracy: {accuracy:.4f}")



1.   모델을 학습 모드로 설정
2.   미니배치마다 예측, 손실 계산, 역전파, 파라미터 업데이트 수행
3.   누적 손실과 정확도 계산 후 출력



In [ ]:
# 6. 테스트 함수
def test(model, loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            output = model(x)
            preds = output.argmax(dim=1)
            correct += (preds == y).sum().item()

    accuracy = correct / len(loader.dataset)
    print(f"Test Accuracy: {accuracy:.4f}")



1.   평가 모드로 변경
2.   기울기 계산 끄고 테스트 데이터셋에 대해 정확도 측정
3.   테스트 정확도 출력



In [ ]:
# 7. 메인 학습 루프
epochs = 5
for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    train(model, train_loader)
    test(model, test_loader)



1.   지정한 에폭만큼 학습과 테스트를 반복
2.   매 에폭마다 학습 손실, 학습 정확도, 테스트 정확도 출력

